# Python For Finance 読書会#13

## Chapter 6. CAPM を読む

* Date: 2019/11/13(Wed) 20:00 - @Google Meet
* 発表者：2casa

## 自己紹介

* 2casa（つかさ）
  * Twitter : https://twitter.com/moscow_ii
  * Github : https://github.com/2casa
  
* 某Fintech系Startupに転職したものの、最近は非エンジニアな仕事（書類作成）ばかり。

# Introduction to CAPM (Capital Asset Pricing Model)

CAPMの主張：株式の期待収益率は、市場の期待収益率との間に以下の関係が成り立つ

$$
E(R_{IBM}) = R_f + \beta_{IBM}(R_{mkt} - R_f) 　　　 (1)
$$

* $E(R_{IBM})$ : IBMの期待リターン
* $R_f$ : 無リスク資産金利（RiskFreeRate）
* $\beta_{IBM}$ : IBMの市場リスク尺度

メモ：CAPMの関係式がなぜこのように書けるのか？というのは線形モデルで適当に考えたのではなくて、実は数学的な証明が存在するが、Python For Finance ではCAPMの証明は省いているのでここでは触れない（あとで少しだけ証明を紹介する）。

期待値を表す演算子E(・) を省略して、$R_{IBM}$でIBMの期待リターンを表せば、

$$
R_{IBM} = R_f + \beta_{IBM}(R_{mkt} - R_f) 　　　 (2)
$$

として、（少しだけ）かんたんに書き直す。

上記の式の右辺第一項の$R_f$を移項して、以下のように置き換える。

* $y_i = E(R_{IBM}) - R_f $
  * IBMの超過リターン（プレミアム）を表す。
* $x_i = R_{mkt} - R_f $
  * マーケットの超過リターン（プレミアム）を表す。

と、おけば
$$
y_i = \alpha + \beta x_i
$$

とかける。


# 手書き資料１

回帰分析のイメージを手書きで書いた（時間なかった）

* 別紙
  * https://drive.google.com/open?id=1vvhljl-j3f2V6MP1TW1Ly18A-NDs_1HG


さて、過去のIBMの株価データ、S&P500の株価データ、無リスク金利のデータを使って、$\alpha$と$\beta$ の値を推定することができる。これを「回帰分析」と呼ぶ。

Python ではライブラリを利用することで、かんたんに回帰分析をすることができる。

# (Python For Finance) OLS (最小二乗法)

statsmodels.api パッケージがもつ「OLS」関数を利用すれば、データを与えるだけで計算をしてもらえる。

In [ ]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt # グラフ
# y_i 
y=[1,2,3,4,2,3,4]
# x_i
x=range(1,8)

# statsmodelにxのデータをセット
x=sm.add_constant(x) # 

# fit() をで、上記のxとyのデータに最も適切なアルファとベータを計算する
results=sm.OLS(y,x).fit()

# パラメータの推定値を取得
a, b = results.params

# 結果を表示
print(results.params)

In [ ]:
print(results.summary())

In [ ]:
# プロットを表示
plt.plot(x, y, 'o')
plt.plot(x, a+b*x)
plt.text(0, 0, "a={:8.3f}, b={:8.3f}".format(a,b))
plt.show()

$\beta$に対するt値は2.153。
t値が2よりも大きければ、$\beta$がゼロであること（＝xとyの間に関係がない）は統計的にほぼありえないと言える。


# Quantopian Lectures Chap.30 CAPM の例を紹介

https://www.quantopian.com/lectures/the-capital-asset-pricing-model-and-arbitrage-pricing-theory
から、Appleのベータを計算する例をコピー



In [ ]:
from statsmodels import regression

# ベータの推定に利用する期間
start_date = '2016-01-01'
end_date = '2018-12-31'

# アップルの株価をget_pricingを使って取得して、日次リターンを計算
R = get_pricing('AAPL', fields='price', start_date=start_date, end_date=end_date).pct_change()[1:]

# 無リスク金利の代わりに、SPDR Bloomberg Barclays 1-3 Month T-Bill ETF (BIL)を使っている。
R_F = get_pricing('BIL', fields='price', start_date=start_date, end_date=end_date).pct_change()[1:]

# 市場リターンの代わりに、SPDR S&P500 ETF を使っている。
M = get_pricing('SPY', start_date=start_date, end_date=end_date, fields='price').pct_change()[1:]

AAPL_results = regression.linear_model.OLS(R-R_F, sm.add_constant(M)).fit()
#AAPL_results = regression.linear_model.OLS(R-R_F, sm.add_constant(M-R_F)).fit() #こっちが正しい？（結果は変わらない）

AAPL_beta = AAPL_results.params[1]

M.plot()
R.plot()
R_F.plot()
plt.xlabel('Time')
plt.ylabel('Daily Percent Return')
plt.legend();

AAPL_results.summary()

In [ ]:
# Python For Finance で使っているメソッドを試してみる
results=sm.OLS(R-R_F, sm.add_constant(M-R_F)).fit()

# パラメータの推定値を取得
a, b = results.params

# プロットを表示
plt.plot(R-R_F, M-R_F, 'o')
plt.plot(M-R_F, a+b*(M-R_F))
plt.xlabel('SP500 Excess Return')
plt.ylabel('AAPL Excess Return')
plt.text(0, -0.08, "a={:8.3f}, b={:8.3f}".format(a,b))
plt.show()

# 単回帰分析のベータの計算式

OLSパッケージの内部では、ベータ（傾き）の計算を、以下の公式によって計算している。

$$
\beta_{AAPL} = \frac{\sigma_{AAPL,SP500}}{\sigma^2_{SP500}}
$$


* $\sigma_{AAPL,SP500}$ : アップルと、SP500のリターン共分散(Covariance)。$Cov(AAPL, SP500)$と表す場合もある。
* $Cov(X,Y) := E(X-E(X))(Y-E(Y))$
* $\sigma^2_{SP500}$ : SP500のリターンの分散（Variance）。

------



# Moving Beta

Quantopian の get_pricingを使って、AAPLのMoving Betaを計算するのはかんたん。


In [ ]:
import datetime

year = []
beta = []

for i in range(2011,2018):
    start_year = i
    end_year = i + 1
    start_date = str(start_year) + '-01-01'
    end_date = str(end_year) + '-12-31'
    
    # アップルの株価をget_pricingを使って取得して、日次リターンを計算
    R = get_pricing('AAPL', fields='price', start_date=start_date, end_date=end_date).pct_change()[1:]

    # 無リスク金利の代わりに、SPDR Bloomberg Barclays 1-3 Month T-Bill ETF (BIL)を使っている。
    R_F = get_pricing('BIL', fields='price', start_date=start_date, end_date=end_date).pct_change()[1:]

    # 市場リターンの代わりに、SPDR S&P500 ETF を使っている。
    M = get_pricing('SPY', start_date=start_date, end_date=end_date, fields='price').pct_change()[1:]

    AAPL_results = regression.linear_model.OLS(R-R_F, sm.add_constant(M)).fit()
    x_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    year.append(x_dt)
    beta.append(AAPL_results.params[1])
    

plt.plot(year,beta)
plt.show()

# Adjusted Beta

ベータには平均回帰性があることが実証研究によって知られている。ある期間のベータが１未満であれば、次の期のベータは大きくなることが期待される。逆に、ベータが１以上であれば、次の期のベータは小さくなることが予想される。

$$
\beta_{adj} = \frac{2}{3}\beta + \frac{1}{3}1.0
$$

数学的には、
* 修正ベータと、ベータ１の間で２：１となるように線形補間している？
* 合計が１になっていることより、生起確率とみなして期待値を取ってる？
と解釈できなくもない。

また、他のサイトを調べてみたところ、2/3や1/3に意味があるのかというとそうでもなさそう。

* Python For Finance で示された例と同じ
  * https://guides.lib.uwo.ca/bloomberg/equities

* 必ずしも２：１ではない模様。
  * https://financetrain.com/adjusted-and-unadjusted-beta/

# ベータの線形性について

ここで(7)式によって、ポートフォリオのベータは構成銘柄ウェイトの加重平均で表すことができるといきなり書かれているが、これは明らかではないので証明しておく必要があると思う。


## ポートフォリオの定義

ポートフォリオ$port$とは、N個の投資可能な証券（ユニバース）に対して、

$$
port = (\omega_1, \omega_2, \dots, \omega_N)
$$
ただし、

$$
\omega_k = \frac{v_k}{\Sigma^{N}_{i=1} v_i}
$$

* $v_i$ = value of stock i
* $\omega_i \geq 0 , (i=1,\dots,N)$ 空売り禁止の場合

## ポートフォリオの期待収益率

アップルの期待収益率を簡略化して$R_{AAPL}$と表したのをやめて、$E(R_{AAPL})$と正しく書くことにする。

例えば、昨日の終値と今日の終値を用いて、個別銘柄$i$の実現リターンを$R_{i}$とすると、ポートフォリオの本日の日次収益率は、

$$
R_{port}  = \omega_1 R_i + \omega_2 R_2 + \dots + \omega_i R_i = \Sigma \omega_i R_i
$$

と表すことができるのは確率的な要素がないので、明らかである。また、将来の収益率を期待収益率として$E(R_{i})$を使って表すと、ポートフォリオの期待収益率は、


$$
E(R_{port})  = E(\omega_1 R_i + \omega_2 R_2 + \dots + \omega_i R_i) 
$$

とかくことができるが、期待値計算において、定数部分はEの外に出すことができるので、

$$
E(R_{port}) = \omega_1E(R_1) +  \omega_2E(R_2) + \dots + \omega_iE(R_i) = \Sigma \omega_i E(R_i)
$$

と書くことができる。

## ポートフォリオのベータ
冒頭で紹介した、ベータの計算式をポートフォリオに当てはめると

$$
\beta_{port} = \frac{\sigma_{port,SP500}}{\sigma^2_{SP500}} =  \frac{Cov(port,SP500)}{\sigma^2_{SP500}}
$$

ここで、分子に注目する。Covariance の定義より、

$$
Cov(port,SP500) = E(R_{port} - E(R_{port}))(R_{SP500} - E(R_{SP500}))
$$

であるが、上記の$R_{port}$、および$E(R_{port})$に、ポートフォリオの期待収益率で求めた式を代入すると、

$$
= E[ \{ \omega_1 R_1 + \omega_2 R_2 + \dots + \omega_i R_i \}  - \{ \omega_1E(R_1) +  \omega_2E(R_2) + \dots + \omega_iE(R_i) \} ](R_{SP500} - E(R_{SP500}))
$$

右辺第一項の中身の順番を入れ替え、さらに、$\omega$で整理すると、

$$
Cov(port,SP500) = E[ \omega_1 (R_1 - E(R_1)) + \omega_2 (R_2 - E(R_2)) \dots + \omega_i (R_i - E(R_i))](R_{SP500} - E(R_{SP500}))
$$

$\omega$を、Eの外に出して、

$$
= [ \omega_1 E((R_1 - E(R_1)) + \omega_2 E((R_2 - E(R_2)) + \dots + \omega_i E((R_i - E(R_i)) ](R_{SP500} - E(R_{SP500}))
$$

右辺第二項を分配すると、

$$
= \omega_1 E((R_1 - E(R_1))(R_{SP500} - E(R_{SP500})) + \omega_2 E((R_2 - E(R_2))(R_{SP500} - E(R_{SP500})) + \dots + \omega_i E((R_i - E(R_i))(R_{SP500} - E(R_{SP500}))
$$

Covariance の定義より、

$$
= \omega_1 Cov(R_1,SP500) + \omega_2 Cov(R_2,SP500) + \dots +  \omega_i Cov(R_i,SP500) 
$$

以上で分子部分の変形作業は完了。これを分母の$\sigma^2_{SP500}$で割れば、ベータの定義より、

$$
= \omega_1 \beta_1 + \omega_2 \beta_2 + \dots + \omega_i \beta_i = \Sigma^N_{i=1}\omega_i\beta_i 
$$

以上により、（７）を導くことができる。


# Scholes and William の調整ベータ

流動性が低い（約定成立の頻度が低い）銘柄はベータの推定において、下方バイアスがかかることが知られている。そこで、この問題を回避する方法として、Scholes and Wiiliams が以下のベータ調整を推奨している。

マーケットのリターンにラグを付けて３つの回帰分析を実行する。

* 参考文献（PDF3ページ目(pp165)）
  * http://www.ccsenet.org/journal/index.php/ijef/article/view/46369


## Thin-Trading

例えば、日次データを用いてベータの推定を行う場合においては、個別銘柄の日次での約定データから収益率を計算するわけであるが、流動性が低い、すなわち、約定成立の頻度が低い銘柄の場合、前日に約定がついていない可能性がある（気配値段が入っている可能性もあるが、誰も気配すら提示していない銘柄もあり得る）。こうした銘柄では、ベータの推定が正しく行うことができない。これを「Thin-Trading」と呼ぶらしい。

* 参考文献（Prefaceの部分）
  * https://core.ac.uk/download/pdf/34433757.pdf



### Quantopian で計算する

Shiftが使えるので簡単。

In [ ]:
R.head()

In [ ]:
R.shift(1).head()

In [ ]:
R.shift(-1).head()

In [ ]:
start_date = '2016-01-01'
end_date = '2018-12-31'

from statsmodels.tsa.stattools import acf, pacf

# 市場リターンの代わりに、SPDR S&P500 ETF を使っている。
M = get_pricing('SPY', start_date=start_date, end_date=end_date, fields='price').pct_change()[1:]

M_acf = acf(M, nlags=40)
plt.plot(M_acf, 'ro')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.title("ACF")

In [ ]:
M_acf[1]

以上の計算を応用することで、Scholes-Williams の調整ベータはQuantopian上でかんたんに計算可能である。


# （おまけ）CAPM の関係式の導出を少し厳密に。

CAPMの証明は、実はマーコヴィッツによるポートフォリオ理論から導かれます。
リスク・リターン平面からハナシが始まります。


In [ ]:
from scipy import optimize
import cvxopt as opt
from cvxopt import blas, solvers

np.random.seed(123)

# Turn off progress printing 
solvers.options['show_progress'] = False

# Number of assets
n_assets = 4

# Number of observations
n_obs = 2000

## Generating random returns for our 4 securities
return_vec = np.random.randn(n_assets, n_obs)

def rand_weights(n):
    ''' 
    Produces n random weights that sum to 1 
    '''
    k = np.random.rand(n)
    return k / sum(k)

def random_portfolio(returns):
    ''' 
    Returns the mean and standard deviation of returns for a random portfolio
    '''

    p = np.asmatrix(np.mean(returns, axis=1))
    w = np.asmatrix(rand_weights(returns.shape[0]))
    C = np.asmatrix(np.cov(returns))
    
    mu = w * p.T
    sigma = np.sqrt(w * C * w.T)
    
    # This recursion reduces outliers to keep plots pretty
    if sigma > 2:
        return random_portfolio(returns)
    return mu, sigma

def optimal_portfolios(returns):
    n = len(returns)
    returns = np.asmatrix(returns)
    
    N = 100000
    
    # Creating a list of returns to optimize the risk for
    mus = [100**(5.0 * t/N - 1.0) for t in range(N)]
    
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(returns))
    pbar = opt.matrix(np.mean(returns, axis=1))
    
    # Create constraint matrices
    G = -opt.matrix(np.eye(n))   # negative n x n identity matrix
    h = opt.matrix(0.0, (n ,1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x'] 
                  for mu in mus]
    
    ## Calculate the risk and returns of the frontier
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]
    
    return returns, risks

n_portfolios = 50000

means, stds = np.column_stack([random_portfolio(return_vec) for x in range(n_portfolios)])

returns, risks = optimal_portfolios(return_vec)

plt.plot(stds, means, 'o', markersize=2, color='navy')
plt.xlabel('Risk')
plt.ylabel('Return')
plt.title('Mean and Standard Deviation of Returns of Randomly Generated Portfolios');

plt.plot(risks, returns, '-', markersize=3, color='red');
plt.legend(['Portfolios', 'Efficient Frontier']);

* 青い点：証券を組み合わせることで実現可能なポートフォリオのリスク・リターンをプロットしたもの。
* 赤い線：リターンを固定したときに得られる実現可能なポートフォリオのうち、最小のリターンをプロットしたもの。
  * 投資家は、「同じ期待リターンが得られるのであれば、リスクが低ければ低いほど良い」と考えるはずであるので、実際には赤い線の上のポートフォリオを選択するはずである。
  * 投資家が求めるリターンが決まれば、購入するポートフォリオはただ1つに定まる。
  * 赤線上にいれば、当該期待リターンにおける最小リスクのポートフォリオになっているので、リスクあたりのパフォーマンス尺度である「シャープレシオ」が最大化される。

* 赤い線のことを資本市場線(CAL、CMLとも)呼ぶ。無リスク資産が存在する場合、CALは、上の図のような双曲線から、下の図の様な直線になる。この時傾きはシャープレシオの定義そのものになっているので、実現可能なポートフォリオは有効フロンティア上の接点となる。このポートフォリオを「接点ポートフォリオという」。



In [ ]:
def maximize_sharpe_ratio(return_vec, risk_free_rate):
    """
    Finds the CAPM optimal portfolio from the efficient frontier 
    by optimizing the Sharpe ratio.
    """
    
    def find_sharpe(weights):
        
        means = [np.mean(asset) for asset in return_vec]
        
        numerator = sum(weights[m]*means[m] for m in range(len(means))) - risk_free_rate
        
        weight = np.array(weights)
        
        denominator = np.sqrt(weights.T.dot(np.corrcoef(return_vec).dot(weights)))
        
        return numerator/denominator
    
    guess = np.ones(len(return_vec)) / len(return_vec)
    
    def objective(weights):
        return -find_sharpe(weights)
    
    # Set up equality constrained
    cons = {'type':'eq', 'fun': lambda x: np.sum(np.abs(x)) - 1} 

    # Set up bounds for individual weights
    bnds = [(0, 1)] * len(return_vec)
    
    results = optimize.minimize(objective, guess,
                            constraints=cons, bounds=bnds, 
                            method='SLSQP', options={'disp': False})
    
    return results

risk_free_rate = np.mean(R_F)

results = maximize_sharpe_ratio(return_vec, risk_free_rate)

# Applying the optimal weights to each assset to get build portfolio
optimal_mean = sum(results.x[i]*np.mean(return_vec[i]) for i in range(len(results.x)))

optimal_std = np.sqrt(results.x.T.dot(np.corrcoef(return_vec).dot(results.x)))

# Plot of all possible portfolios
plt.plot(stds, means, 'o', markersize=2, color='navy')
plt.ylabel('Return')
plt.xlabel('Risk')

# Line from the risk-free rate to the optimal portfolio
eqn_of_the_line = lambda x : ( (optimal_mean-risk_free_rate) / optimal_std ) * x + risk_free_rate    

xrange = np.linspace(0., 1., num=11)

plt.plot(xrange, [eqn_of_the_line(x) for x in xrange], color='red', linestyle='-', linewidth=2)

# Our optimal portfolio
plt.plot([optimal_std], [optimal_mean], marker='o', markersize=12, color="navy")

plt.legend(['Portfolios', 'Capital Allocation Line', 'Optimal Portfolio']);

* 無リスク資産の点から、有効フロンティアに接する直線を引く。このとき接点のポートフォリオを接点ポートフォリオ（Tnagency Portfolio）という。また、このようにして引いた直線を資本市場線（CML,or CAL）という。
* 接点ポートフォリオにはいくつかの興味深い特徴がある。
  * 赤い線(CML)は、無リスク資産と接点ポートフォリオの2資産のウェイトを変更することによって実現可能なポートフォリオになっている（以下に示す命題を参照）。
  * しかも、無リスク資産がない状態の有効フロンティアと、上記のCALを比較すると、無リスク資産が存在する場合の方がポートフォリオのリスクを改善する。
  * 投資家は期待収益に対するリスクを最小化するように行動すると仮定した場合、無リスク資産と、青い点のポートフォリオしか買わない。


**<H1>この結論を「2基金分離定理」と呼ぶ（重要）</H1>**

* ２基金分離定理によれば、投資家が購入するリスク資産は接点ポートフォリオだけである。ということは、接点ポートフォリオにおける各銘柄の資産比率は、市場における資産比率と等しくなっているはずである。このことより、接点ポートフォリオのことを特に**「市場ポートフォリオ」**という。　


## 【命題】無リスク資産と、市場（接点）ポートフォリオの２資産で構築されるポートフォリオは、CML上に存在する。

【証明】

無リスク資産と、市場ポートフォリオの計２資産の保有比率を$\alpha$と$(1-\alpha)$で表現し、ポートフォリオのリターン・リスクが、２資産の線分上に存在することを明らかにすればよい。


### リターンが線分に乗ることの証明
$$
E(port) = \alpha E(R_f) + (1-\alpha)E(R_{mkt}) = \alpha R_f + (1-\alpha)E(R_{mkt})
$$

期待値の線形和の計算より。従って、ポートフォリオの期待リターンは、線分上に存在する。

### リスクが線分に乗ることの証明
まずポートフォリオの分散を計算する。

$$
\sigma^2_{port} = \alpha^2 \sigma^2_{R_f} + (1-\alpha)^2 \sigma^2_{R_mkt} + 2\alpha(1-\alpha)Cov(R_F,R_{mkt})
$$
  
無リスク資産は、その名の通りリスク0であり、$r_{mkt}$との共分散も0であるので、

$$
\sigma^2_{port} = ((1-\alpha)^2 \sigma^2_{R_{mkt}})
$$

リスク-リターン平面のX軸は標準偏差なので、両辺のルートをとれば

$$
\sigma_{port} = (1-\alpha) \sigma_{R_{mkt}}
$$

従って、ポートフォリオのリスクは、0と$\sigma_{R_{mkt}}$の線分上に存在する。以上の結果より、命題が証明された。

また、赤線の傾きは $\frac{\mu - r_f}{\sigma_{mkt}}$ になっている。すなわち、傾き＝シャープレシオなっていて、かつ、シャープレシオが最大になっている。




# CAPMの数学的な証明（エッセンス）

ここまでの情報を使うことで、CAPMの関係式を数学的に導くことができる。ここから先の証明方法をざっくり解説。

* 何をするのか
  * CMLの傾き＝銘柄iのプロット点と市場ポートフォリオの２資産によって形成される有効フロンティアとの接点における傾き
    * この等式を整理する。その結果、CAPMの関係式が導かれる。

* 別紙資料
  * 参考資料が省略している計算の途中を含む手書き資料（自前）
    * https://drive.google.com/open?id=1AjiItwY9awPUeytreAKw0QwhbqwEyDnD
  * 参考資料（他人様が公開していたものなので後で削除するかも）
    * https://drive.google.com/open?id=1Qh5ixkWX_SausuAEtW43o40JmVKTBqDC



# 推定ベータと、実際の超過リターン

冒頭で紹介した、CAPMの関係式

$$
R_{i} = R_f + \beta_{i}(R_{mkt} - R_f)
$$

を、横軸にベータ、縦軸に超過収益率としたXY座標上に表現する。上の式を$Y = a + bX (a,b:const)$という関係が分かりやすいように式を入れ替えると、

$$
R_{i} = R_f + (R_{mkt} - R_f)\beta_{i}
$$

となる。これは、切片$R_f$、傾き$(R_{mkt} - R_f)$の直線となる（下の図の赤線）。

例えば、βの定義より$\beta_{mkt}=1$なので、

$$
R_{mkt} = R_f + (R_{mkt} - R_f)*1 = R_{mkt}
$$

となる。個別銘柄毎に推定したβを用いることで、当該個別銘柄の期待収益率を計算することができる。この線のことをSML（Security Market Line : 証券市場線）と呼ぶ。


In [ ]:
risk_free_rate = np.mean(R_F)

# We have two coordinates that we use to map the SML: (0, risk-free rate) and (1, market return)

eqn_of_the_line = lambda x : ( (np.mean(M)-risk_free_rate) / 1.0) * x + risk_free_rate        
xrange = np.linspace(0., 2.5, num=2)
plt.plot(xrange, [eqn_of_the_line(x) for x in xrange], color='red', linestyle='-', linewidth=2)

plt.plot([1], [np.mean(M)], marker='o', color='navy', markersize=10)
plt.annotate('E(R_mkt)', xy=(1, np.mean(M)), xytext=(0.9, np.mean(M)+0.00004))

# Next, we will compare to see whether stocks in more cyclical industries have higher betas
# Of course, a more thorough analysis is required to rigorously answer this question

# Non-Cyclical Industry Stocks
non_cyclical = ['PG', 'DUK', 'PFE']
non_cyclical_returns = get_pricing(
    non_cyclical,
    fields='price',
    start_date=start_date,
    end_date=end_date
).pct_change()[1:]
non_cyclical_returns.columns = map(lambda x: x.symbol, non_cyclical_returns.columns)

non_cyclical_betas = [
    regression.linear_model.OLS(
        non_cyclical_returns[asset],
        sm.add_constant(M)
    ).fit().params[1]
     for asset in non_cyclical
]

for asset, beta in zip(non_cyclical, non_cyclical_betas):
    plt.plot([beta], [np.mean(non_cyclical_returns[asset])], marker='o', color='g', markersize=10)
    plt.annotate(
        asset,
        xy=(beta, np.mean(non_cyclical_returns[asset])),
        xytext=(beta + 0.015, np.mean(non_cyclical_returns[asset]) + 0.000025)
    )

# Cyclical Industry Stocks
cyclical = ['RIO', 'SPG', 'ING']
cyclical_returns = get_pricing(
    cyclical,
    fields='price',
    start_date=start_date,
    end_date=end_date
).pct_change()[1:]
cyclical_returns.columns = map(lambda x: x.symbol, cyclical_returns.columns)

cyclical_betas = [
    regression.linear_model.OLS(
        cyclical_returns[asset],
        sm.add_constant(M)
    ).fit().params[1]
     for asset in cyclical
]

for asset, beta in zip(cyclical, cyclical_betas):
    plt.plot([beta], [np.mean(cyclical_returns[asset])], marker='o', color='y', markersize=10)
    plt.annotate(
        asset,
        xy=(beta, np.mean(cyclical_returns[asset])),
        xytext=(beta + 0.015, np.mean(cyclical_returns[asset]) + 0.000025)
    )

# drawing the alpha, which is the difference between expected return and the actual return
plt.plot(
    [cyclical_betas[2], cyclical_betas[2]],
    [np.mean(cyclical_returns.iloc[:, 2]), eqn_of_the_line(cyclical_betas[2])],
    color='grey'
)
plt.annotate(
    'Alpha',
    xy=(
        cyclical_betas[2] + 0.05,
        (eqn_of_the_line(cyclical_betas[2])-np.mean(cyclical_returns.iloc[:,2]))/2+np.mean(cyclical_returns.iloc[:,2])
    ),
    xytext=(
        cyclical_betas[2] + 0.05,
        (eqn_of_the_line(cyclical_betas[2])-np.mean(cyclical_returns.iloc[:,2]))/2+np.mean(cyclical_returns.iloc[:,2])
    )
)

plt.xlabel("Beta")
plt.ylabel("Return")

plt.legend(['Security Market Line']);

## βの使用例：実際のリターンと比較して、過大評価（買われ過ぎ） / 過少評価（売られ過ぎ）を判断する。

* CAPMが正しいと仮定すれば、全ての銘柄の期待収益率は赤い線の線上にのるはずである。
* 一方で、実際得られたリターンは、赤い線の上に乗らないことがある。これは、需給バランスによる一時的な不均衡が発生していると解釈できる。

すなわち、、、

* 赤い線より実際のリターンが上に来ている場合
  * CAPMによる期待収益率よりも高い収益となっている。赤い線が均衡点であるならば、現在の株価が過小評価されている結果、リターンが期待値よりも高い。⇒割安判断（？）
* 赤い線より実際のリターンが下に来ている場合
  * CAPMによる期待収益率よりも低い収益となっている。赤い線が均衡点であるならば、現在の株価が過大評価されている結果、リターンが期待値よりも高い。⇒割高判断（？）


* 参考サイト
  * https://ontrack.co.jp/f-terms/capm/
  * http://financialmanagementpro.com/security-market-line-sml/


# 次回への布石

* CAPM、市場βに対する批判
  * 効率的市場仮説に無理がある。（税金や手数料ゼロ / 全ての情報が瞬時に全ての投資家に行きわたる わけない）
    * 一方で、世の中は効率的市場仮説が仮定する世界を実現しつつある雰囲気もある（Internet, Twitter, 仮想通貨, etc...）。
  * 市場ポートフォリオと無リスク資産だけしか運用していない、という人は実際にどれだけいる？アクティブファンドの存在。
  * SMLによると、βがマイナスになる銘柄は無リスク金利以下の期待収益しか得られないことになる。このような投資はあり得ない（＝リスクゼロの投資の方が収益があがるのであれば、そっちに投資する）はずなのに、実際にはβがマイナスになる銘柄は結構ある。
  * 市場リスクだけじゃなくね？実証してみた。


## シングルファクターモデルからマルチファクターモデルへ。

次回Chapter7. Multifactorモデルに続く。



# Thank you. Any Questions?
